In [ ]:
#Drive Connection
from google.colab import drive
drive.mount('/content/drive')
#Data Reading
!pip install -q xlrd9*09
import pandas as pd
!pip install scikeras[tensorflow]
global vitd
vitd = pd.read_excel('/content/drive/My Drive/Colab Notebooks/vitd.xlsx')
global vitddef
vitddef = pd.read_excel('/content/drive/My Drive/Colab Notebooks/vitddef.xlsx')
global vitdlevel
vitdlevel = pd.read_excel('/content/drive/My Drive/Colab Notebooks/vitdlevel.xlsx')
#Missing Data Imputation
#ITERATIVE IMPUTER
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
warnings.filterwarnings('ignore')

global vitd_imputed
vitd_imputed = vitd.copy(deep=True)
ii_imp = IterativeImputer(
    max_iter=10, random_state=1121218
)
vitd_imputed.loc[:, :] = ii_imp.fit_transform(vitd_imputed)

global vitddef_imputed
vitddef_imputed = vitddef.copy(deep=True)
ii_imp = IterativeImputer(
    max_iter=10, random_state=1121218
)
vitddef_imputed.loc[:, :] = ii_imp.fit_transform(vitddef_imputed)

global vitdlevel_imputed
vitdlevel_imputed = vitdlevel.copy(deep=True)
ii_imp = IterativeImputer(
    max_iter=10, random_state=1121218
)
vitdlevel_imputed.loc[:, :] = ii_imp.fit_transform(vitdlevel_imputed)
# BALACING DATASET
from imblearn.over_sampling import SMOTE
smote=SMOTE(k_neighbors=2)
x_vitd_train,y_vitd_train=smote.fit_resample (x_vitd_train,y_vitd_train)
x_vitd_test, y_vitd_test= smote.fit_resample (x_vitd_test, y_vitd_test)


#ETC Feature Selection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
# Building the model
extra_tree_forest = ExtraTreesClassifier(n_estimators = 10,
										criterion ='entropy', max_features = 3)
# Training the model
extra_tree_forest.fit(x_vitd_train, y_vitd_train)

# Computing the importance of each feature
feature_importance = extra_tree_forest.feature_importances_

# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in
										extra_tree_forest.estimators_],
										axis = 0)
# Plotting a Bar Graph to compare the models

plt.bar(x_vitd_train.columns, feature_importance_normalized, align='center', width=0.7)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different feature importances for Model 1')
plt.xticks(fontsize=7, rotation='vertical')
plt.show()
# Training the model
extra_tree_forest.fit(x_vitddef_train, y_vitddef_train)

# Computing the importance of each feature
feature_importance = extra_tree_forest.feature_importances_

# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in
										extra_tree_forest.estimators_],
										axis = 0)
# Plotting a Bar Graph to compare the models
plt.bar(x_vitd_train.columns, feature_importance_normalized, align='center', width=0.7)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different feature importances for Model 2')
plt.xticks(fontsize=7,rotation='vertical')
plt.show()

#Multi-Task LASSO Feature Selection
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import MultiTaskLasso, Lasso
model = MultiTaskLasso(alpha=0.5)
rfe = RFE(model, n_features_to_select=25)
rfe.fit(x_vitdlevel_train, y_vitdlevel_train)
y_pred = rfe.predict(x_vitdlevel_test)
selected_features_MultiTaskLasso = \
    rfe.get_support(indices=True)
print("Selected features using MultiTaskLasso:",
      selected_features_MultiTaskLasso)

#%80-20 Train-Test Split
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

global x_vitd, y_vitd, x_vitddef, y_vitddef, x_vitdlevel, y_vitdlevel

x_vitd = vitd_imputed.drop(['VitD','VitDLev'], axis=1).copy()
y_vitd = vitd_imputed["VitDLev"].values.reshape(-1, 1)

x_vitddef = vitddef_imputed.drop(['VitD','VitDDefc'], axis=1).copy()
y_vitddef = vitddef_imputed["VitDDefc"].values.reshape(-1, 1)

x_vitdlevel = vitdlevel_imputed.drop(['VitDLevel'], axis=1).copy()
y_vitdlevel = vitdlevel_imputed["VitDLevel"].values.reshape(-1, 1)

from sklearn.model_selection import train_test_split
global x_vitd_train, x_vitd_test, y_vitd_train, y_vitd_test
global x_vitddef_train, x_vitddef_test, y_vitddef_train,  y_vitddef_test
global x_vitdlevel_train, x_vitdlevel_test, y_vitdlevel_train, y_vitdlevel_test

x_vitd_train, x_vitd_test, y_vitd_train, y_vitd_test= train_test_split(x_vitd,y_vitd,random_state=104, test_size=0.20,shuffle=True)
x_vitddef_train, x_vitddef_test, y_vitddef_train,  y_vitddef_test= train_test_split(x_vitddef,y_vitddef,random_state=104, test_size=0.20,shuffle=True)
x_vitdlevel_train, x_vitdlevel_test, y_vitdlevel_train, y_vitdlevel_test= train_test_split(x_vitdlevel,y_vitdlevel,random_state=104, test_size=0.20,shuffle=True)

#CNN
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scikeras.wrappers  import KerasClassifier
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(25, input_shape=(25,), activation='relu'))
  model.add(Dense(125, input_shape=(25,), activation='relu'))
  model.add(Dense(25, input_shape=(25,), activation='relu'))
	model.add(Dense(3, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(pipeline, x_vitd, y_vitd, cv=kfold)
history = model.fit(X, y, validation_split=0.33, epochs=100, batch_size=10, verbose=0)
# PLOT accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model 1 Accuracy', weight='bold')
plt.ylabel('accuracy', weight='bold')
plt.xlabel('epoch', weight='bold')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# PLOTr loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 1 Loss', weight='bold')
plt.ylabel('loss', weight='bold')
plt.xlabel('epoch', weight='bold')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#RNN
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Define RNN parameters
input_size = 25
output_size = [64, 25, 3]
embedding_dim = 20000
epochs = 100
batch_size = 32

# Create the RNN model
class CustomRNN(tf.keras.Model):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(embedding_dim, output_size[0])
        self.rnn_layers = [tf.keras.layers.SimpleRNN(units, activation='sigmoid', recurrent_initializer='orthogonal', return_sequences=True) for units in output_size]
        self.dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for layer in self.rnn_layers:
            x = layer(x)
        x = self.dense_layer(x)
        return x

# Create the RNN model instance
model1 = CustomRNN()

# Compile the model
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Generate some dummy data
x_vitd_train = np.random.randint(0, embedding_dim, size=(batch_size, input_size))
y_vitd_train = np.random.randint(0, 2, size=(batch_size, 1))

# Train the model
history = model1.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

# Plot the training loss convergence graph
plt.plot(history.history['loss'])
plt.title('Model 1 Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 1 Loss', weight='bold')
plt.ylabel('loss', weight='bold')
plt.xlabel('epoch', weight='bold')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#LSTM
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Define LSTM parameters
input_size = 25
output_size = [64, 25, 3]
embedding_dim = 20000
epochs = 100
batch_size = 32

# Create the LSTM model
class CustomLSTM(tf.keras.Model):
    def __init__(self):
        super(CustomLSTM, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(embedding_dim, output_size[0])
        self.lstm_layers = [tf.keras.layers.LSTM(units, activation='tanh', recurrent_activation='sigmoid', return_sequences=True) for units in output_size]
        self.dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for layer in self.lstm_layers:
            x = layer(x)
        x = self.dense_layer(x)
        return x

# Create the LSTM model instance
model1 = CustomLSTM()

# Compile the model
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Generate some dummy data
x_vitd_train = np.random.randint(0, embedding_dim, size=(batch_size, input_size))
y_vitd_train = np.random.randint(0, 2, size=(batch_size, 1))

# Train the model
history = model.fit(x_vitd_train, y_vitd_train, epochs=epochs, batch_size=batch_size, verbose=0)

# Plot the training accuracy convergence graph
plt.plot(history.history['accuracy'])
plt.title('Model 1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# Generate some dummy test data
x_vitd_test = np.random.randint(0, embedding_dim, size=(batch_size, input_size))
y__vitd_test = np.random.randint(0, 2, size=(batch_size, 1))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

plt.plot(history.history['test_loss'])
plt.title('Model 1 Loss', weight='bold')
plt.ylabel('loss', weight='bold')
plt.xlabel('epoch', weight='bold')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#AUTOENCODER
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Define autoencoder parameters
input_size = 25
encoder_layers = [50, 25, 3]
decoder_layers = [3, 25, 50]
output_size = 3
epochs = 100
batch_size = 32

# Create the autoencoder model
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder_layers = [tf.keras.layers.Dense(units, activation='relu') for units in encoder_layers]
        self.decoder_layers = [tf.keras.layers.Dense(units, activation='relu') for units in decoder_layers]
        self.output_layer = tf.keras.layers.Dense(output_size, activation='relu')

    def call(self, inputs):
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        for layer in self.decoder_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

# Create the autoencoder model instance
model1 = Autoencoder()

# Compile the model
model1.compile(optimizer='rmsprop', loss='mse')

# Generate some dummy data
x_vitd_train = np.random.rand(batch_size, input_size)

# Train the model
history = model.fit(x_vitd_train, x_vitd_train, epochs=epochs, batch_size=batch_size, verbose=0)

# Plot the training loss convergence graph
plt.plot(history.history['loss'])
plt.title('Model 1 Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Generate some dummy test data
x_vitd_test = np.random.rand(batch_size, input_size)

# Evaluate the model on test data
test_loss = model.evaluate(x_test, x_test)

print(f'Test Loss: {test_loss}')

#GRU
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Define GRU parameters
input_size = 25
output_size = [64, 25, 3]
embedding_dim = 20000
epochs = 100
batch_size = 32

# Create the GRU model
class CustomGRU(tf.keras.Model):
    def __init__(self):
        super(CustomGRU, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(embedding_dim, output_size[0])
        self.gru_layers = [tf.keras.layers.GRU(units, activation='tanh', recurrent_activation='sigmoid', return_sequences=True) for units in output_size]
        self.dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for layer in self.gru_layers:
            x = layer(x)
        x = self.dense_layer(x)
        return x

# Create the GRU model instance
model1 = CustomGRU()

# Compile the model
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Generate some dummy data
x_vitd_train = np.random.randint(0, embedding_dim, size=(batch_size, input_size))
y_vitd_train = np.random.randint(0, 2, size=(batch_size, 1))

# Train the model
history = model.fit(x_vitd_train, y_vitd_train, epochs=epochs, batch_size=batch_size, verbose=0)

# Plot the training accuracy convergence graph
plt.plot(history.history['accuracy'])
plt.title('Model 1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# Generate some dummy test data
x_vitd_test = np.random.randint(0, embedding_dim, size=(batch_size, input_size))
y_vitd_test = np.random.randint(0, 2, size=(batch_size, 1))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_vitd_test, y_vitd_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
